In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Echarts Jupyter lab support
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [ ]:
# Echarts imports
from pyecharts import options as opts
from pyecharts.globals import  ThemeType
from pyecharts.charts import Line

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt

In [ ]:
from common.log_helper import LogHelper
from backtest.scale_backtest_args import ScaleBacktestArgGen
from backtest.scale_chain_reader import ScaleChainReader
from notebooks.img_symbols import down_arrow
from notebooks.echart_df_plot import plot_df_simple
from modelservice.smoothers import EMA, Whittaker
import logging
import sys
import pandas as pd
import talib

In [ ]:
LogHelper.configure_logging(verbose=False)
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
ReaderArgs = {
    'common': {
        'start_time': '09:15:30',
        'end_time' : '09:30:15',
    },
}

In [ ]:
dump_base_dir = "/home/qoptrprod/data/dumps"
trading_date = "20240320"
underlying = "MICDPNIFTY_I"

In [ ]:
arg_obj = ScaleBacktestArgGen.resolve_and_gen_args(dump_base_dir, underlying, trading_date, ReaderArgs)
reader = ScaleChainReader(arg_obj)

In [ ]:
analysis_recs = []
ema = EMA(args={'timeperiod' : 60 })
whit = Whittaker(args={'lmbda' : 10, 'qlen' : 60 })

for chain in reader.read_chain():
    ts = chain.get_ul_exch_ts()
    ulpx = chain.get_ul_price()
    futpx = chain.get_trd_ltp()
    
    ema.add_sample(ulpx)
    inema = ema.get_value()

    whit.add_sample(ulpx)
    inwhit = whit.get_value()
    
    analysis_recs.append({
        'ts' : ts,
        'ulpx' : ulpx,
        'futpx' : futpx,
        'inema' : inema,
        'inwhit' : inwhit,
    })
df = pd.DataFrame(analysis_recs)
_ = df.set_index('ts')

In [ ]:
print(analysis_recs[0])

In [ ]:
df['sma'] = talib.SMA(df.ulpx, timeperiod=60)
df['ema'] = talib.EMA(df.ulpx, timeperiod=60)

In [ ]:
df_plot = plot_df_simple(df, 'ts', 
                         ['ulpx' , 'futpx', 'sma', 'ema', 'inema', 'inwhit'], 
                         title= 'Analysis')

In [ ]:
df_plot.render_notebook()